[View in Colaboratory](https://colab.research.google.com/github/fmacrae/AI-Learning/blob/master/DCGAN_Credit_Card_Data.ipynb)

###SETUP
Connect to Google Drive to store the model and access data.

1.   If setting this to false watch out for discconections to the VM - It's epherial though so you'll lose all your checkpoints
2.   Also define how much training to do

Based on the work by Arthur Juliani, check out his other work! https://github.com/awjuliani/TF-Tutorials 

Changes:

1.   Integrated with Google Drive to allow storage of model checkpoints and sample outputs
2.   Moved onto Collabatory - Make sure you select runtime with GPU
3.   Improved resiliancy by allowing it to restart from last checkpoint
4.   Moved setup and training options to top for easier setup.
5.   Reworking to use credit card data from https://www.kaggle.com/mlg-ulb/creditcardfraud

Useful Links:  

https://github.com/soumith/ganhacks

https://sigmoidal.io/beginners-review-of-gan-architectures/

https://www.toptal.com/machine-learning/generative-adversarial-networks







In [0]:
use_cloud_storage = True ## change to false if you want to just store the images and checkpoints on the colab VM. Will need to get the cc data on manually though 
iterations = 1001 #Total number of iterations to use.  If you go lower than 1001 then no checkpoint is stored.


In [0]:
if use_cloud_storage:
   !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
   !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
   !apt-get update -qq 2>&1 > /dev/null
   !apt-get -y install -qq google-drive-ocamlfuse fuse
   from google.colab import auth
   auth.authenticate_user()
   from oauth2client.client import GoogleCredentials
   creds = GoogleCredentials.get_application_default()
   import getpass
   !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
   vcode = getpass.getpass()
   !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
data_file_train = 'creditcard-train.csv'
data_file_validation = 'creditcard-validation.csv'
data_file_predict = 'creditcard-predict.csv'


if use_cloud_storage:
  !mkdir -p drive
  !google-drive-ocamlfuse drive
  sample_directory = './drive/ccsamples' #Directory to save sample output from generator in.
  model_directory = './drive/ccmodels' #Directory to save trained model to.
  data_directory = './drive/datasets' #Directory to save trained model to.

else:
  sample_directory = './ccsamples' #Directory to save sample output from generator in.
  model_directory = './ccmodels' #Directory to save trained model to.
  data_directory = './datasets' #Directory to save trained model to.

data_full_path_train = data_directory + '/' + data_file_train
data_full_path_validation = data_directory + '/' + data_file_validation
data_full_path_predict = data_directory + '/' + data_file_predict

# Deep Convolutional Generative Adversarial Network (DCGAN) Tutorial

This tutorials walks through an implementation of DCGAN as described in [Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks](https://arxiv.org/abs/1511.06434).

To learn more about generative adversarial networks, see my [Medium post](https://medium.com/p/54deab2fce39) on them.

In [0]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
#from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

Prepare and load the dataset into what tensorflow will accept - 

Using guide from here:  https://learningtensorflow.com/ReadingFilesBasic/ 

and 

https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/examples/tutorials/input_fn/boston.py

Data looks like this:
*   Time -  Number of seconds elapsed between this transaction and the first transaction in the dataset
*   V1  -  V1 to V28 are all anonymised data features that are roughly converted into a real number between -1 and 1.  Not exactly normalised with mean of 0 but close.
*   V2
*   V3
*   V4
*   V5
*   V6
*   V7
*   V8
*   V9
*   V10
*   V11
*   V12
*   V13
*   V14
*   V15
*   V16
*   V17
*   V18
*   V19
*   V20
*   V21
*   V22
*   V23
*   V24
*   V25
*   V26
*   V27
*   V28  
*   Amount -  Transaction amount
*   Class - 1 for fraudulent transactions, 0 otherwise

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd


COLUMNS = ["Time", "V1", "V2", "V3",
"V4", "V5", "V6", "V7",
"V8", "V9", "V10", "V11",
"V12", "V13", "V14", "V15",
"V16", "V17", "V18", "V19",
"V20", "V21", "V22", "V23",
"V24", "V25", "V26", "V27",
"V28", "AmountTransaction",
"Class"]

FEATURES = ["Time", "V1", "V2", "V3",
"V4", "V5", "V6", "V7",
"V8", "V9", "V10", "V11",
"V12", "V13", "V14", "V15",
"V16", "V17", "V18", "V19",
"V20", "V21", "V22", "V23",
"V24", "V25", "V26", "V27",
"V28", "AmountTransaction"]

LABEL = "Class"


def get_input_fn(data_set, num_epochs=None, shuffle=True):
  return tf.estimator.inputs.pandas_input_fn(
      x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
      y=pd.Series(data_set[LABEL].values),
      num_epochs=num_epochs,
      shuffle=shuffle)

training_set = pd.read_csv(data_full_path_train, skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

test_set = pd.read_csv(data_full_path_validation, skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

prediction_set = pd.read_csv(data_full_path_predict, skipinitialspace=True,
                             skiprows=1, names=COLUMNS)



Check the data looks OK by printing the second row for the datafile

In [0]:
print(training_set.iloc[[2]])

   Time        V1        V2        V3       V4        V5        V6        V7  \
2     1 -1.358354 -1.340163  1.773209  0.37978 -0.503198  1.800499  0.791461   

         V8        V9  ...         V21       V22       V23       V24  \
2  0.247676 -1.514654  ...    0.247998  0.771679  0.909412 -0.689281   

        V25       V26       V27       V28  AmountTransaction  Class  
2 -0.327642 -0.139097 -0.055353 -0.059752             378.66      0  

[1 rows x 31 columns]


Let's train a basic classifier to see if we can realiably predict Class for fraud.

In [0]:
  # Feature cols
  feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

  # Build 2 layer fully connected DNN with 10, 10 units respectively.
  regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                        hidden_units=[10, 10],
                                        model_dir=model_directory)

  # Train
  regressor.train(input_fn=get_input_fn(training_set), steps=5000)

  # Evaluate loss over one epoch of test_set.
  ev = regressor.evaluate(input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))
  loss_score = ev["loss"]
  print("Loss: {0:f}".format(loss_score))

  # Print out predictions over a slice of prediction_set.
  y = regressor.predict(input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
  
  # .predict() returns an iterator of dicts; convert to a list and print
  # predictions
  predictions = list(p["predictions"] for p in itertools.islice(y, 6))
  print("Predictions: {}".format(str(predictions)))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc9e66764d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './drive/ccmodels', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./drive/ccmodels/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:ten

### Helper Functions

In [0]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [0]:
def generator(z):
    
    zP = slim.fully_connected(z,4*4*256,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,256])
    
    gen1 = slim.convolution2d_transpose(\
        zCon,num_outputs=64,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(\
        gen1,num_outputs=32,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(\
        gen2,num_outputs=16,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(\
        gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [0]:
def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,16,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,32,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,64,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis3),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

### Connecting them together

In [0]:
tf.reset_default_graph()

z_size = 100 #Size of z vector used for generator.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These two placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images

Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gz,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.

tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!
Check out the output in your Google Drive in the folder figs to see how it progresses.

In [0]:
batch_size = 128 #Size of image batch to apply at each iteration.


init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    #might be resuming a crashed training session so try loading the model
    try:
        print 'Loading Model...'
        ckpt = tf.train.get_checkpoint_state(model_directory)
        saver.restore(sess,ckpt.model_checkpoint_path)
    except:
        print 'model didnt load, OK to ignore on first run'

    for i in range(iterations):
        zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        xs,_ = mnist.train.next_batch(batch_size) #Draw a sample batch from MNIST dataset.
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 32x32
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs}) #Update the discriminator
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs}) #Update the generator, twice for good measure.
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs})
        if i % 100 == 0:
            print "Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss)
            z2 = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate another z batch
            newZ = sess.run(Gz,feed_dict={z_in:z2}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            save_images(np.reshape(newZ[0:36],[36,32,32]),[6,6],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print "Saved Model"

Loading Model...
INFO:tensorflow:Restoring parameters from ./drive/models/model-49000.cptk
Gen Loss: 1.8926408 Disc Loss: 0.41932273
Gen Loss: 2.5666914 Disc Loss: 0.4712001
Gen Loss: 2.7967997 Disc Loss: 0.25285047
Gen Loss: 1.9136124 Disc Loss: 0.7580761
Gen Loss: 2.8225193 Disc Loss: 1.0394459
Gen Loss: 1.8651094 Disc Loss: 0.293338
Gen Loss: 2.0234199 Disc Loss: 0.40296856
Gen Loss: 2.236509 Disc Loss: 0.44627184
Gen Loss: 2.382635 Disc Loss: 0.42547378
Gen Loss: 1.3726362 Disc Loss: 0.84920204
Gen Loss: 1.0347916 Disc Loss: 0.651014
Saved Model


## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [0]:
batch_size_sample = 36

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    #Reload the model.
    print 'Loading Model...'
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    zs = np.random.uniform(-1.0,1.0,size=[batch_size_sample,z_size]).astype(np.float32) #Generate a random z batch
    newZ = sess.run(Gz,feed_dict={z_in:z2}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    save_images(np.reshape(newZ[0:batch_size_sample],[36,32,32]),[6,6],sample_directory+'/sample'+str(i)+'.png')

Loading Model...
INFO:tensorflow:Restoring parameters from ./drive/models/model-1000.cptk
